# GIS U-Net

A Workflow for applying a Convolutional Neural Network to Geospatial data.
Input data is a multi layer GeoTiff.
Output data is a geotiff, this demo is semantic segmentation, however regression is also possible.



## How to use
>Google Colab lets you run interactive python scripts (Jupyter Notebooks).
>This demo has been set up to run by simply executing all cells one after another.
>To run each cell press Shift-Enter
>
>If you do not want to train the network skip the *"Training"* section.
>If you want to train the network from scratch, skip the cell that loads the pre-trained model.
>
>
>## Data Prep
>
> ### Raster generation
>
>Lidar data is converted into multiple raster layers. We used LasTools. Specifically LasCanopy.
>The Lidar point cloud (pcl) was flattened to move all ground points to 0m.
>The resulting flattened pcl is split into slices at differing heights above ground. rasters where generated using relative point densisty for ground layer ( # of points in slice/ # of total points)  and normalized point density(# of points in slice/ # of points in and below slice) with a grid of 4m.
>for the canopy and upper vegetation structure, percentile heights where used with a 1m grid.
>
>The resulting rasters should be merged into a single Geotiff (future updates may allow for independent files)
>
>
> ### Training / Test Data
>![Input Tile](https://github.com/tpet93/GIS_Neural_Network/raw/master/Images/InputTile.png) ![Training Tile](https://github.com/tpet93/GIS_Neural_Network/raw/master/Images/TrainingTile.png) 
>
>Training / Test data is created by drawing polygons using GIS software.
>Each polygon is labeled with a class (integer)
>two attributes "Training_Class" and "Test_Class" are used to split polygons between training and test.
>
>
> ### Google Drive
>It is recommended to mount and store datasets in a google drive folder, this will allow automatic saving of snapshots and output data to a persistent storage location.





In [0]:
!nvidia-smi
#shows which GPU has been assigned.
#T4 is significantly faster than K80 due to mixed precision support.

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


## **Setup Environment**



In [0]:
'''Optional. If datasets are stored on google drive use this option (recomended)'''

#gdrive is used to store snapshots of the network and is a good place to store large datasets for quick access


# from google.colab import drive
# drive.mount('/content/gdrive',force_remount=True)



In [0]:
#@title Install dependencys , this can take some time. { output-height: 40, display-mode: "form" }


!pip install tifffile

! rm -rf /root/.ssh/*
! mkdir /root/.ssh
!ssh-keyscan github.com >> /root/.ssh/known_hosts 

! git clone https://github.com/NVIDIA/apex
% cd apex
! pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./
% cd ..

#!pip install tb-nightly  # Until 1.14 moves to the release channel
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

!wget https://raw.githubusercontent.com/postmates/gdal/master/scripts/gdal_merge.py -P /content/ -nc

In [0]:
#@title Import Python Modules

import os
import torch
import numpy as np

import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torchvision import datasets
from torchsummary import summary
import torchvision.utils as vutils
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

try:
    from apex import amp
    APEX_AVAILABLE = True
except ModuleNotFoundError:
    APEX_AVAILABLE = False
print(APEX_AVAILABLE)

True


In [0]:
!wget https://www.dropbox.com/sh/4kzz3fcyhno3qxc/AACOKj2gDZTK019FpNrD24EYa?dl=1
!unzip AACOKj2gDZTK019FpNrD24EYa?dl=1
!unzip 9band_lidar_slices.zip  #Tifffile library seems to have issue with compressed tiffs when using memmap mode. zipping  the tiff as a work around


In [0]:
#@title Get Modules from Github.

!rm -rf GIS_Neural_Network

!git clone https://github.com/tpet93/GIS_Neural_Network.git

from GIS_Neural_Network.Modules import dataloaders as dl
from GIS_Neural_Network.Modules import utils as ut
from GIS_Neural_Network.Modules import loaders as ld
from GIS_Neural_Network.Modules import models as models
from GIS_Neural_Network.Modules import train as tr
from GIS_Neural_Network.Modules import classify as cl

import imp
imp.reload(dl)
imp.reload(ut)
imp.reload(ld)
imp.reload(tr)
imp.reload(cl)
imp.reload(models)# needs to be last?


In [0]:
#@title Start TensorBoard
LOG_DIR = '/content/runs/'

get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR) 
)

get_ipython().system_raw('./ngrok http 6006 &')
get_ipython().system(' curl -s http://localhost:4040/api/tunnels | python3 -c     "import sys, json; print(json.load(sys.stdin)[\'tunnels\'][0][\'public_url\'])"')


http://24c55db5.ngrok.io


## **Generate Files**




In [0]:
#@title Parameter input

# Path Containing the Training Data

color_file = '/content/9band_lidar-slices.tif'
class_file = '/content/Ground_truth_polygons.gpkg'



#number of U-Net Blocks
depth = 4

#Calculate padsize and input image dimensions 
neat = ut.find_neats(36,depth)
isneat, ps = ut.calc_padsize(depth,neat)
print(neat,isneat,ps)
#this keeps the downscaling of our images to a even divisible number for each layer,it is not neccesesary but seems to be good practise.


#Calculate image dimensions for final classifaction
fullneat = ut.find_neats(128,depth)
fisneat, ps = ut.calc_padsize(depth,neat)
print(fullneat,fisneat,ps)


#save image Dimensions
tile_size = neat
full_tile_size = fullneat

#number of Classes to classifiy (+1 for ignore class 0)
nc = 11


# Define the bands to use in the input image and how to display them

#9 band standard

num_bands = 9

#disable any special band selection
bandtransform = None
# def bandtransform(img):
#     img = img[:,:,(1,2,3,4,5,6)]# only use understory layers
#     return img


num_bands = 9

#show bands 1,2, and 0 as RGB channels
bands = (1,2,0)
#scale RGB channels to 0-1 (overflow is acceptable)
dividers = [100,100,30]



# #9 band exaggerated percentile
# num_bands = 9

# def bandtransform(img):
#     img[:,:,(7,8,0)] = img[:,:,(7,8,0)]*100#this multiplies the canopy heights by 3 to bring them close to the 0-100 scale that percentiles have
#     return img

# bands = (1,-1,0)
# dividers = [100,25,4000]



#each region can be tiled multiple times with a random offset to increase the size of the the training data.
#each tile already has a buffer around it to account for the cropping.
#to disable set spt to 1 and  maxshiftsize to 0.


# num of shifts per tile
spt = 5
#the amount a tile can be shifted by
maxshiftsize = tile_size/2
# maxshiftsize = 0


#nullthresh  is the cut-off propertion to discard tiles that contain little or no classified pixels.

#if countall = True then blackthresh will be a percentage of the pixels, 
#if False then the the value used is the average of: the number of rows and the number of columns that contain at least one non-zero pixel. 
#this is helpful for tiles containg long thin classes such as roads

countall = False

nullthresh = 0.01 #proportion of pixels to be labled in each training tile
te_nullthresh = 0.01 #proportion of pixels to be labled in each test tile

# an image to genereate a test output (best to user a smaller image than full dataset)
# used to confirm settings are appropriate / working
test_infolder = '/content/'
test_infile = '9band_lidar-slices.tif'

# the full image to genereate an output
full_infolder = '/content/'
full_infile = '9band_lidar-slices.tif'

# a folder on the VM to store tiles/outputs in.
workdir = '/content/Segmentation/'

# where to store training tiles
tr_color_folder = 'tr_Color/'
tr_class_folder = 'tr_Class/'

# where to store eval tiles
te_color_folder = 'te_Color/'
te_class_folder = 'te_Class/'

# where to store the input full and testing tiles
full_folder = 'Full/'
test_folder = 'Test/'

# where to store the generated full and testing tiles
predicted_test_folder = 'Predicted_test/'
predicted_full_folder = 'Predicted_full/'

#where to store the model periodically
checkpoint_path = '/content/GIS_Neural_Network/Models/'
checkpoint_path = '/content/GIS_Neural_Network/'

#where to store the hdf5 datasets

tr_datasetpath = '/content/train/'
te_datasetpath = '/content/eval/'



348 True 44.0
1084 True 44.0


In [0]:
'''
------Caution------
run this cell to remove all files, this will allow the next cell to generate new training tiles 

'''
!rm -rf $workdir # Remove tiles to generate new ones

In [0]:
#@title Generate Tiles from gpkg


#each block can be commeented out depending on requirements

if not os.path.exists(workdir):
    os.mkdir(workdir)
    print("Directory " , workdir ,  " Created ")

    #generate Training Tiles
    print('Generate training tiles')
    dl.gen_trainingtilesfromgpkg(
        colourfile = color_file,
        classfile = class_file,
        attribute = 'Training_Class',
        workdir = workdir,
        color_folder = tr_color_folder,
        class_folder = tr_class_folder,
        tilesize = tile_size,
        maxshift = maxshiftsize,
        shiftspertile = spt,
        ps = ps,
        nullthresh = nullthresh,
        countall = countall)

    #generate Evaluation Tiles
    print('Generate evaluation tiles')
    dl.gen_trainingtilesfromgpkg(
        colourfile =  color_file,
        classfile = class_file,
        attribute = 'Test_Class',
        workdir = workdir,
        color_folder = te_color_folder,
        class_folder = te_class_folder,
        tilesize = tile_size,
        maxshift = maxshiftsize,
        shiftspertile = spt,
        ps = ps,
        nullthresh = te_nullthresh,
        countall = countall)

    #generate Test Tiles

    print('Generate test dataset tiles')
    dl.gen_fulltiles(
        input_folder = test_infolder,
        colorfile = test_infile,
        workdir = workdir,
        output_folder = test_folder,
        tilesize = full_tile_size,
        ps = ps)
    
#     generate Full Tiles, Comment the next block during training, testing to speed up.
    print('Generate test full dataset tiles')
    dl.gen_fulltiles(
        input_folder = full_infolder,
        colorfile = full_infile,
        workdir = workdir,
        output_folder = full_folder,
        tilesize = full_tile_size,
        ps = ps)

    #Split the Full tiles down the middle to create artifical boundarys.
    print('split training tiles')
    dl.split_shuffle(
        color_input_folder = workdir+tr_color_folder,
        class_input_folder = workdir+tr_class_folder,
        ps = ps,
        nullthresh = 0.25)

In [0]:
#@title Make Hdf5 datafiles

dl.makehdf5(tr_datasetpath,workdir+tr_color_folder, workdir+tr_class_folder,int(ps),bandtransform = bandtransform,lbltransform = None)
dl.makehdf5(te_datasetpath,workdir+te_color_folder, workdir+te_class_folder,int(ps),bandtransform = bandtransform,lbltransform = None)

## Build Model

In [0]:
#@title Define Model
#This cell also reset the model to a blank state
if(APEX_AVAILABLE):
    amp_handle = amp.init(enabled=True)

model = models.UNetB(n_classes=nc, padding=False, up_mode='upsample',in_channels=num_bands,wf=5,batch_norm=True,activation = 'ELU',depth=depth,max_filters= 512).to(device)
writer = SummaryWriter()
torch.cuda.empty_cache()
epoch = 0
summary(model, input_size=(num_bands, tile_size, tile_size))

In [0]:
#@title Load Per-Trained Model (skip this cell to train from scratch)

ckpt = torch.load('/content/GIS_Neural_Network/Models/Unet-Lidar-9b-11C-ELU-d4wf5-140-Ev_Loss-0.834403.pth')
optimizer = optim.Adam(model.parameters())

# print(ckpt['state_dict'])
model.load_state_dict(ckpt['state_dict'])
optimizer.load_state_dict(ckpt['opt_state_dict'])
epoch = ckpt['epochs']


In [0]:
#@title convert the model to amp (mixed precision)
optimizer3 = optim.Adam(model.parameters(), lr=1e-3,weight_decay = 1e-2)
optimizer4 = optim.Adam(model.parameters(), lr=1e-4,weight_decay = 1e-2)
optimizer5 = optim.Adam(model.parameters(), lr=1e-5,weight_decay = 1e-2)
optimizer6 = optim.Adam(model.parameters(), lr=1e-6,weight_decay = 1e-2)
optimizer6 = optim.Adam(model.parameters(), lr=1e-6,weight_decay = 1e-6)
optimizer7 = optim.Adam(model.parameters(), lr=1e-7)

#convert the model to amp (mixed precision)
model, [optimizer3, optimizer4, optimizer5, optimizer6, optimizer7] = amp.initialize(model,  [optimizer3, optimizer4, optimizer5, optimizer6, optimizer7],opt_level="O1")#setup our optimizers and run the model through AMP.


##**Training**


skip forward to Generate Outputs to generate map from pretrained model

In [0]:
#@title Define training factors matrix

#this this matric allow us to configure the output class based on all the class probailitys.
#this changes the prediciton strenghts for each class before picking the maximum.
#i.e sav = np.matrix([0,0,1,0,0,0,.7,.7,.7,0]) would add the predicitons of classes 7,8,9
# into the prediction value of class 3 before pick the class with the maximum prediciton value.


pri = np.array([1,0,0,0,0,0,0,0,0,0])
sec = np.array([0,1,0,0,0,0,0,0,0,0])
sav = np.array([0,0,1,0,0,0,0,0,0,0])
isl = np.array([0,0,0,1,0,0,0,0,0,0])
ww =  np.array([0,0,0,0,1,0,0,0,0,0])    
stu = np.array([0,0,0,0,0,1,0,0,0,0])
hea = np.array([0,0,0,0,0,0,1,0,0,0])
roa = np.array([0,0,0,0,0,0,0,1,0,0])
ear = np.array([0,0,0,0,0,0,0,0,1,0])
riv = np.array([0,0,0,0,0,0,0,0,0,1])

factsmat = np.stack([pri,sec,sav,isl,ww,stu,hea,roa,ear,riv])

#pad the matrix to account for the 0 class
factsmat = np.pad(factsmat, (1,0), 'constant', constant_values=0)#add ignore class columns and rows
factsmat[0][0]=1
# print(factsmat)




In [0]:
#@title Setup datasets

batch_size = 20 #this is limited by GPU memory, (due to using mixed precision the memory usage will be far lower than that shown in the model summary )


#print the number of tiles
train_samples = len(os.listdir(workdir+tr_color_folder))
eval_samples = len(os.listdir(workdir+te_class_folder))

print ('Training Tiles: ',train_samples)
print ('Eval Tiles: ',eval_samples)


#optional transform for adding random noise to input image
data_transforms = transforms.Compose([
    transforms.Lambda(lambda x : torch.from_numpy(x.copy())),
    transforms.Lambda(lambda x : torch.transpose(x,0,2)),#image gets flipped going from numpy to tensor
    transforms.Lambda(lambda x : torch.transpose(x,1,2)),
    transforms.Lambda(lambda x : x + torch.randn_like(x)*(0.06*100))# add noise with a mean of 0 and variance of 6
    ])

#set datatransforms to None  to ignore
data_transforms =None


traindataset = ld.HDF5Dataset(tr_datasetpath,data_transforms)
evaldataset = ld.HDF5Dataset(te_datasetpath,None)


loader_params = {'batch_size': 1, 'shuffle': False, 'num_workers': 1}
class_data_loader = data.DataLoader(traindataset, **loader_params) # a loader to count all the pixels in each class, this loader has a batch size of 1

loader_params = {'batch_size': batch_size, 'shuffle': True, 'num_workers': 1}
train_data_loader = data.DataLoader(traindataset, **loader_params)

loader_params = {'batch_size': batch_size, 'shuffle': True, 'num_workers': 1}
eval_data_loader = data.DataLoader(evaldataset, **loader_params)


Training Tiles:  1389
Eval Tiles:  650


In [0]:
#@title Get class weights

#Counts the number of pixels and applies the inverse to account for unbalanced classes
train_class_weights = ut.get_class_weights(class_data_loader,nc)

#Adjust the weights depending on the importance of the class for our purposes
train_class_weights =train_class_weights * [0,1,1,1,1.2,0.0,.8,.6,.6,.6,.6]

# train_class_weights =train_class_weights * [0,1,1,1,1,1,1,1,1,1,1]

print(train_class_weights)

[ 0.          1.40833171  1.          1.74881432  2.87873158  0.
 12.55532015 17.00379108 20.77867377 23.58905551 12.68446643]


In [0]:
#@title Initialize training routine

#initialize our training routine
trainer = tr.train_model(
    model = model,
    device = device,
    epoch = epoch,
    train_loader = train_data_loader,
    eval_loader = eval_data_loader,
    image_path = workdir+tr_color_folder,
    class_path = workdir+tr_class_folder,
    evalimage_path = workdir+te_color_folder,
    evalclass_path = workdir+te_class_folder,
    writer = writer,# tensorboard Summary writer
    checkpoint_path = checkpoint_path, # where to save the model
    weightsmat = factsmat, 
    ps = ps , # number of pixels that are cropped on the output
    bands = bands,  # bands to display in previews
    dividers = dividers, # bring the bands to usefull value to display (0-1)
    use_amp = True, #use automatic mixed precision
    amp = amp,
    label = 'Lidar-9b-11C-ELU-d4wf5', # the Label to Save the Model Files Under
    print_every = 1, # print text output every (epoch)
    show_every = 1, # show sample image  every (epoch)
    eval_every = 1, # run eval dataset every (epoch)
    save_every = 10, # save model every (epoch)
    asses_every = 4) # every (n batches) asses precision and jaccard) slows down training but provides useful graphs
    
    
criterion = nn.CrossEntropyLoss(weight = torch.FloatTensor(train_class_weights).to(device), ignore_index=0)



In [0]:
#@title show sample images
ld.show_image(model,device,factsmat,eval_data_loader,ps,bands,dividers)

ld.show_image(model,device,factsmat,train_data_loader,ps,bands,dividers)

In [0]:
#@title Restart Tensorboard
#ngrok will die when the training is interupted, run this cell to resart it

get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://f98dc402.ngrok.io


Train network with decreasing learning rate

In [0]:
trainer(criterion, optimizer3, num_epochs=15)


In [0]:
trainer(criterion, optimizer4, num_epochs=100)

In [0]:
trainer(criterion, optimizer5, num_epochs=100)

In [0]:
trainer(criterion, optimizer6, num_epochs=50)

## **Generate Outputs**

In [0]:
#@title Define Final factors Matrix

#here we can remove, replace, or reduce the prevalence of certain classes

pri = np.array([1,0,0,0,0,0,0,0,0,0])
sec = np.array([0,1,0,0,0,0,0,0,0,0])
sav = np.array([0,0,1,0,0,0,0,0,0,0])
isl = np.array([0,0,0,0.6,0,0,0,0,0,0])
ww =  np.array([0,0,0,0,0,0,0,0,0,0])    
stu = np.array([0,0,0,0,0,.6,0,0,0,0])
hea = np.array([0,0,0,0,0,0,.8,0,0,0])
roa = np.array([0,0,0,0,0,0,0,.7,0,0])
ear = np.array([0,0,0,0,0,0,0,0,.7,0])
riv = np.array([0,0,0,0,0,0,0,0,0,1])


factsmat = np.stack([pri,sec,sav,isl,ww,stu,hea,roa,ear,riv])

factsmat = np.pad(factsmat, (1,0), 'constant', constant_values=0)#add ignore class columns and rows
factsmat[0][0]=1
print(factsmat)

In [0]:
#@title Setup Dataloaders

data_transforms = None

# def bandtransform(img):#pick the same subset of bands as used for training
#     img = img[:,:,(1,2,3,4,5,6)]# only use understory layers
#     return img

testdataset = ld.tifimagedataset(workdir+test_folder,bandtransform)

fulldataset = ld.tifimagedataset(workdir+full_folder,bandtransform)

loader_params = {'batch_size': 1, 'shuffle': False, 'num_workers': 1}
test_data_loader = data.DataLoader(testdataset, **loader_params)
full_data_loader = data.DataLoader(fulldataset, **loader_params)



In [0]:
#@title Generate Test Map
#test and full map are the same in this demo, 
#the test dataset is entended to be a smaller region
#to allow quick testing of parameters before processing the full map.

cl.generate_image2(model,device,test_data_loader,workdir,test_folder,predicted_test_folder,factsmat,ps,output = 'raw',average = True)

filename = 'test_map.tif'
command = cl.merge(workdir+predicted_test_folder,workdir,filename)
! $command

In [0]:
#@title Generate Full Map

cl.generate_image2(model,device,full_data_loader,workdir,full_folder,predicted_full_folder,factsmat,ps,output = 'raw',average = True)
#genrerate
filename = 'full_map.tif'
command = cl.merge(workdir+predicted_full_folder,workdir,filename)
! $command

In [0]:
#@title Copy Final Map to Google Drive

#google drive must be mounted to avoid error.

infile = workdir+filename
!mv $infile "/content/gdrive/My Drive/GIS/"